In [62]:
import pandas as pd

import os 
PATH = '/home/politlen/Downloads/Telegram Desktop'
history = 'history.csv'
states = 'states.csv'

hist_df = pd.read_csv(os.path.join(PATH, history), converters={'history': eval})

In [63]:
len(hist_df)

1099

In [64]:
import json
import datetime

pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)

test_cell = hist_df.history.loc[0]

def get_updates(json):
    return [x for x  in json if x['type']=='update']
    

def to_dt(string):
    return datetime.datetime.strptime(string, '%Y-%m-%d_%H:%M:%S')

def get_time_diff(json):
    return to_dt(json[-1]['ts']) - to_dt(json[0]['ts'])

def get_data(json):
    length = len(json)
    
    return length

print(get_data(test_cell))
# print(json.dumps(get_updates(test_cell), sort_keys=True, indent=4, ensure_ascii=False))
print(get_time_diff(test_cell))
hist_df['len'] = hist_df.history.apply(lambda x:len(x))
hist_df['updates'] = hist_df.history.apply(lambda x:len(get_updates(x)))
hist_df['hello'] = hist_df.history.apply(lambda x: len([i for i in x if i['action'] == 'hello']))
hist_df['start'] = hist_df.history.apply(lambda x: len([i for i in x if i['action'] == '/start']))

hist_df['time_diff'] = hist_df.history.apply(get_time_diff)
hist_df.columns

36
0:06:32


Index(['Unnamed: 0', 'user_hash', 'history', 'len', 'updates', 'hello', 'start', 'time_diff'], dtype='object')

In [ ]:
hist_df.history.loc[5]

In [66]:
low = hist_df[hist_df.time_diff < datetime.timedelta(seconds=30)]
print('Количество запией: ', len(low))
low.head()

Количество запией:  626


,Unnamed: 0,user_hash,history,len,updates,hello,start,time_diff
11,11,fca6afefbea8d8b531c4f13d7f994c2623713891b77825...,"[{'type': 'update', 'action': '/start', 'ts': ...",12,5,1,1,00:00:12
13,13,f52e928aeb540b0e1a7f1572b6ad4dabb6bf239954a047...,"[{'type': 'update', 'action': '/start', 'ts': ...",3,1,1,1,00:00:00
14,14,166d7f828381afd0980a64550f2729bc97f47232054911...,"[{'type': 'update', 'action': '/start', 'ts': ...",7,3,1,1,00:00:09
15,15,31ec8e61e3bc60129ad36cb079712e62529e4e9dfd9d97...,"[{'type': 'update', 'action': '/start', 'ts': ...",3,1,1,1,00:00:00
16,16,5aebb54a547059fe784a86101e5f4bec4c282b9d110c77...,"[{'type': 'update', 'action': '/start', 'ts': ...",3,1,1,1,00:00:00


In [ ]:
for i in low.history:
    print(json.dumps(i, sort_keys=True, indent=4, ensure_ascii=False), end = '**'*16)

In [74]:
def get_action_list(df):
    action_list = []
    for row in df:
        action_list.append([x['action'] for x in row if x['type'] == 'update'])
    
    return action_list

In [45]:
import numpy as np

def levenshtein(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
#     print(matrix)
    return (matrix[size_x - 1, size_y - 1])



In [ ]:
lev = []
for seq in get_action_list(hist_df.history):
    if len(seq) == 1:
        lev.append(None)
    else:
        lev.append(levenshtein(seq, ['hello', 'help_callback']))
    
print(lev)

In [51]:
hist_df['lev'] = lev

In [ ]:
for i in hist_df[hist_df.lev<3].history:
    print(json.dumps([x['action'] for x in i if x['type']=='update'], sort_keys=True, indent=4, ensure_ascii=False), end = '\n')


In [ ]:
#подсчет новых и уникальных юзеров за период времени
def to_dt(string):
    if '"' in string:
        string = string.replace('"', '')
    return datetime.datetime.strptime(string, '%Y-%m-%d_%H:%M:%S')

def get_last_date(days=1):
    """days устанавливает значения в днях для получения нужной даты"""
    current_time = datetime.datetime.today() 
    return current_time - datetime.timedelta(days=days)


def get_new_users(array, flag=0):
    """set 0 to get new users and -1 to get unique"""
    counter = 0
    for elem in array:
        date = to_dt([x['ts'] for x in elem][flag])
        if date>get_last_date(1):
            counter +=1
    return counter


def get_new_users_indexes(array, flag=0):
    """set 0 to get new users and -1 to get unique"""
    index_list = []
    for index, elem in enumerate(array):
        date = to_dt([x['ts'] for x in elem][flag])
        if date>get_last_date(1):
            index_list.append(index)
            
    return index_list

unique = get_new_users_indexes(hist_df.history, -1)
new = get_new_users_indexes(hist_df.history, 0)
old = set(unique).difference(set(new))
hist_df[hist_df['Unnamed: 0'].isin(unique)]
hist_df[hist_df['Unnamed: 0'].isin(new)]
hist_df[hist_df['Unnamed: 0'].isin(old)]



In [86]:
#кустарный подсчет самых востребованных кнопок
def get_all_updates(df):
    action_list = []
    for row in df:
        action_list.extend([x['action'] for x in row if x['type'] == 'update'])
    
    return action_list


from collections import Counter
cnt = Counter()

for el in get_all_callbacks(hist_df.history):
     el = str(el)
     cnt[el] += 1
        

cnt.most_common(30)

[('/start', 1227),
 ('Полезно знать', 757),
 ('Вернуться', 677),
 ('Советы юристов', 500),
 ('Нет', 470),
 ('Понятно', 431),
 ('Нужна помощь', 315),
 ('Хочу помочь', 231),
 ('Напомнить меры профилактики', 230),
 ('/help', 210),
 ('Мои права нарушают', 139),
 ('Да', 129),
 ('Как не заразиться?', 125),
 ('Я просто тестирую бота', 116),
 ('Когда стоит вызывать врача?', 111),
 ('Выберите свои симптомы.', 109),
 ('Как вести себя при задержании?', 109),
 ('Стать волонтером', 104),
 ('Обнаружил(а) симптомы', 96),
 ('Другая помощь', 91),
 ('Поддержать НКО и местные инициативы', 76),
 ('Хочу помочь нуждающимся', 76),
 ('Что будет, если нарушить карантин?', 67),
 ('Как не заразить окружающих?', 62),
 ('Нет, не разрешаю', 61),
 ('Могут ли меня уволить?', 51),
 ('Да, разрешаю', 44),
 ('Уязвимым людям', 41),
 ('Распространить полезную информацию', 32),
 ('None', 32)]